In [436]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# Import Package
from collections import Counter

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

import math

In [437]:
dataset_x_y_train = pd.read_csv("train.csv")
dataset_x_test = pd.read_csv("test.csv")
dataset_y_test = pd.read_csv("gender_submission.csv")

In [438]:
dataset_x_y_test = dataset_y_test.merge(dataset_x_test, on='PassengerId')

In [439]:
dataset = pd.concat([dataset_x_y_test, dataset_x_y_train])

In [440]:
dataset = dataset.sort_values(['PassengerId'])

In [441]:
print('\nNull Values in data \n{}'.format(dataset.isnull().sum()))
print('\nDuplicated values in data {}'.format(dataset.duplicated().sum()))


Null Values in data 
PassengerId       0
Survived          0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

Duplicated values in data 0


In [442]:
print('Embarkation per ports \n{}'.format(dataset['Embarked'].value_counts()))

Embarkation per ports 
Embarked
S    914
C    270
Q    123
Name: count, dtype: int64


In [443]:
dataset['Embarked'].fillna(value='S', inplace=True)
dataset['Fare'].fillna(value=dataset.Fare.mean(), inplace=True)

In [444]:
mean_age_miss = dataset[dataset["Name"].str.contains('Miss.', na=False)]['Age'].mean().round()
mean_age_mrs = dataset[dataset["Name"].str.contains('Mrs.', na=False)]['Age'].mean().round()
mean_age_mr = dataset[dataset["Name"].str.contains('Mr.', na=False)]['Age'].mean().round()
mean_age_master = dataset[dataset["Name"].str.contains('Master.', na=False)]['Age'].mean().round()

print('Mean age of Miss. title {}'.format(mean_age_miss))
print('Mean age of Mrs. title {}'.format(mean_age_mrs))
print('Mean age of Mr. title {}'.format(mean_age_mr))
print('Mean age of Master. title {}'.format(mean_age_master))

Mean age of Miss. title 22.0
Mean age of Mrs. title 37.0
Mean age of Mr. title 33.0
Mean age of Master. title 5.0


In [445]:
def fill_age(name_age):
    
    name = name_age[0]
    age = name_age[1]
    
    if pd.isnull(age):
        if 'Mr.' in name:
            return mean_age_mr
        if 'Mrs.' in name:
            return mean_age_mrs
        if 'Miss.' in name:
            return mean_age_miss
        if 'Master.' in name:
            return mean_age_master
        if 'Dr.' in name:
            return mean_age_master
        if 'Ms.' in name:
            return mean_age_miss
    else:
        return age

In [446]:
dataset['Age'] = dataset[['Name', 'Age']].apply(fill_age,axis=1)

In [447]:
dataset['Cabin'] = pd.Series(['X' if pd.isnull(ii) else ii[0] for ii in dataset['Cabin']])

In [448]:
print('Mean Fare of Cabin B {}'.format(round(dataset[dataset['Cabin']=='B']['Fare'].mean(), 2)))
print('Mean Fare of Cabin C {}'.format(round(dataset[dataset['Cabin']=='C']['Fare'].mean(), 2)))
print('Mean Fare of Cabin D {}'.format(round(dataset[dataset['Cabin']=='D']['Fare'].mean(), 2)))
print('Mean Fare of Cabin E {}'.format(round(dataset[dataset['Cabin']=='E']['Fare'].mean(), 2)))
print('Mean Fare of Cabin A {}'.format(round(dataset[dataset['Cabin']=='A']['Fare'].mean(), 2)))
print('Mean Fare of Cabin T {}'.format(round(dataset[dataset['Cabin']=='T']['Fare'].mean(), 2)))
print('Mean Fare of Cabin X {}'.format(round(dataset[dataset['Cabin']=='X']['Fare'].mean(), 2)))
print('Mean Fare of Cabin F {}'.format(round(dataset[dataset['Cabin']=='F']['Fare'].mean(), 2)))
print('Mean Fare of Cabin G {}'.format(round(dataset[dataset['Cabin']=='G']['Fare'].mean(), 2)))

Mean Fare of Cabin B 90.92
Mean Fare of Cabin C 72.76
Mean Fare of Cabin D 50.28
Mean Fare of Cabin E 40.41
Mean Fare of Cabin A 38.58
Mean Fare of Cabin T 21.36
Mean Fare of Cabin X 25.29
Mean Fare of Cabin F 25.73
Mean Fare of Cabin G 14.74


In [449]:
def reasign_cabin(cabin_fare):
    
    cabin = cabin_fare[0]
    fare = cabin_fare[1]
    
    if cabin=='X':
        if (fare >= 113.5):
            return 'B'
        if ((fare < 113.5) and (fare > 100)):
            return 'C'
        if ((fare < 100) and (fare > 57)):
            return 'D'
        if ((fare < 57) and (fare > 46)):
            return 'E'
        if ((fare < 46) and (fare > 39)):
            return 'A'            
        else:
            return 'X'
    else:
        return cabin

In [450]:
dataset['Cabin'] = dataset[['Cabin', 'Fare']].apply(reasign_cabin, axis=1)

In [451]:
print('\nNull Values in data \n{}'.format(dataset.isnull().sum()))
print('\nDuplicated values in data {}'.format(dataset.duplicated().sum()))


Null Values in data 
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

Duplicated values in data 0


In [452]:
def create_alone_feature(SibSp_Parch):
    if (SibSp_Parch[0]+SibSp_Parch[1])==0:
        return 1
    else:
        return 0

dataset['Alone'] = dataset[['SibSp','Parch']].apply(create_alone_feature, axis=1)
dataset['Familiars'] = dataset['SibSp'] + dataset['Parch']

In [453]:
categories = {"female": 1, "male": 0}
dataset['Sex']= dataset['Sex'].map(categories)

categories = {"S": 1, "C": 2, "Q": 3}
dataset['Embarked']= dataset['Embarked'].map(categories)

categories = {"X" : 0, "C" : 1, "E" : 2, "G" : 3, "D" : 4, "A" : 5, "B" : 6, "F" : 7, "T" : 8}
dataset['Cabin']= dataset['Cabin'].map(categories)


# dataset['Ticket'] = dataset['Ticket'].astype('category')
# dataset['Ticket'] = dataset['Ticket'].cat.codes

dataset = dataset.drop(['Name','PassengerId'], axis=1)

In [454]:
train = dataset[:891]
test = dataset[891:]

In [455]:
x_train = train.loc[:,['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Alone', 'Familiars']]
y_train = train.loc[:, 'Survived']

x_test = test.loc[:,['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Alone', 'Familiars']]
y_test = test.loc[:, 'Survived']

In [456]:
# Decision Tree Implemention

def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])


class Node:
    def __init__(
        self, feature=None, threshold=None, left=None, right=None, *, value=None
    ):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    # get training data and traning label
    def fit(self, X, y):
        #grow tree
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1]) #ta tedad az kole feature mojoud bishtar nabashe
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        #traverse tree
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape # تعداد نمونه های موجود (تعداد ردیف ها ) و تعداد ویژگی‌های موجود (تعداد ستونها)
        n_labels = len(np.unique(y))  #تعداد لیبل متمایز

        # stopping criteria # متوقف کردن رشد درخت با یکی از شرایط زیر
        if (
            depth >= self.max_depth #عمق فعلی از نهایت عمقی که میتونه باشه پیشی بگیره 
            or n_labels == 1 #همه ی فیچرهایی که موندن همشون دارای لیبل های یکسان هستند یا به عبارتی فقط یک لیبل  موجوده
            or n_samples < self.min_samples_split # تعداد نمونه های موجود یا همون ردیف هایی که مونده از کمترین مقداری که اینجا ۲ تنظیم شده کمتر باشه یعنی یک ردیف بیشتر نداریم
        ):
            leaf_value = self._most_common_label(y) # حالا که ته رشد درخته بیا، اون لیبلی که در اینجا بیشترین تکرار را داره به عنوان برگ برگردون
            return Node(value=leaf_value)

        # اگر درخت باید رشد کند و باید نود جدید ساخته بشه


        #هدف این بخش این هست که به صورت رندم بیاد جایگشتی از فیچرها را در نظر بگیره و ریپلیس اخر برای اینکه هر بار که اجرا میشه در آینده جایگشت تکراری نده
        # should:  (self.n_feats <=  n_features)
        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to information gain
        # هدف اینجا این هست که بهترین فیچر و بهترین تریشهولد اون را برگردونه تا بفهمیم درختو چطوری ریز کنیم
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)

        # grow the children that result from the split
        # حالا که بهترین فیچر برای جداسازی به همراه تریشهولد مناسبش را متوجه شدیم، باید بریم درخت و بر این اساس بشکنیم و بریم پایین
        
        # این خط عملا تقسیم همون چیزی هست که قبلا برای محاسبه‌ی آی.جی حسابش کردیم، و حالا که فهمیدیم این بهترینش بود خب دوباره حسابش میکنیم
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        # اون ردیفهایی که تقسیم شدند تمامی ستونهاشون را میخوایم و لیبلهای مورد نیاز را
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        
        # اون دنباله‌ی رندمی از فیچرها که مشخص کردیم را داریم، حالا هر بار به ترتیب سراغ یکی از فیچر های این دنباله میریم
        for feat_idx in feat_idxs:
            # پس از انتخاب فیچر باید بریم برای انتخاب تریشهولد از بین مقادیر موجود برای اون فیچر(مقادیر اون ستون)
            X_column = X[:, feat_idx]
            # کل مقادیر متمایز اون ستون را پیدا میکنیم و هر بار یکی را به عنوان تریشهولد در نظر میگیریم
            # اینکه چرا مقدار تریشهولد عددی غیر از مقادیر نباشه را نمیدونم مثلا یک عددی بین این مقادیر که موجود نیست
            thresholds = np.unique(X_column)
            # برای هر تریشهولد میره تست کنه تا ببینه کدوم تریشهولد بهترین نتیجه را میده
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        # برای محاسبه باید ببینیم کیا بزرگتر از تریشهولد و کیا کوچیکتر از تریشهولد هستند 
        # خروجی این بخش شماره ردیف اونایی که کوچیکتر از تریشهولد و اونایی که بزرگتر از تریشهولد هستند را برمیگردونه
        # شماره ردیف باعث میشه بتونیم با شماره ردیف به مقدار لیبلشون که در محاسبه اینتروپی نیاز داریم دسترسی بگیریم
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        # اگر براساس رابطه‌ی آی.جی بری اگر یکی از بچه ها خالی باشه در واقع تریشهولد طوری باشه که همه را فرستاده باشه در یکی از بچه ها
        # عملا بر اساس محاسبه خواهیم داشت:
        # E(parent) - 1* E(child)
        # که درعمل بخش دوم دقیقا همان مقدار ای پرنت را خواهد داشت و حاصل برابر با صفر میشود
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        # با ایندکس ها به مقادیر لیبل های اون ایندکسهای مورد نظر دسترسی پیدا کردیم
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common


In [457]:
# Random Forest Implemetion

def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    #result = pd.DataFrame(X.values[idxs], columns=['PassengerId', 'Pclass','Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'])
    result = pd.DataFrame(X.values[idxs], columns=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Alone', 'Familiars'])
    return result, y[idxs]


def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common


class RandomForest:
    def __init__(self, n_trees=10, min_samples_split=2, max_depth=100, n_feats=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            clf = DecisionTreeClassifier(min_samples_split = self.min_samples_split,
                                         max_features = self.n_feats,
                                         max_depth = self.max_depth,
                                         class_weight = "balanced")
            
            X_samp, y_samp = bootstrap_sample(X, y)
            clf.fit(X_samp, y_samp)
            self.trees.append(clf)
        return self.trees

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)

In [458]:
clf = RandomForest(n_trees=100, max_depth=4, n_feats=5)
trees = clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

In [459]:
result = confusion_matrix(y_pred, y_test)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, y_pred)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Confusion Matrix:
[[252   4]
 [ 14 148]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.95      0.97       266
           1       0.91      0.97      0.94       152

    accuracy                           0.96       418
   macro avg       0.95      0.96      0.95       418
weighted avg       0.96      0.96      0.96       418

Accuracy: 0.9569377990430622
